In [1]:
import numpy as np
import pandas as pd
import os
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt
%matplotlib inline
print ("Package loaded")
cwd = os.getcwd()
print ("Current folder is %s" % (cwd) )

Package loaded
Current folder is /Users/psbots/Desktop/hack2innovate


In [2]:
def loadData():
    '''
    Data providing function:

    This function is separated from model() so that hyperopt
    won't reload data for each evaluation run.
    '''
    imgsize = [1080, 1920]

    cwd = os.getcwd()
    loadpath = cwd + "/data/UIScreensProcessedData.npz"
    l = np.load(loadpath)

    # Parse data
    X_train = l['trainimg']
    Y_train = l['trainlabel']
    X_test = l['testimg']
    Y_test = l['testlabel']
    
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)
    X_train = (X_train - mean)/std
    X_train = X_train.reshape(-1,imgsize[0],imgsize[1],3)
    X_train = X_train.reshape(-1,3,imgsize[0],imgsize[1])
    
    mean = np.mean(X_test, axis=0)
    std = np.std(X_test, axis=0)
    X_test = (X_test - mean)/std
    X_test = X_test.reshape(-1,imgsize[0],imgsize[1],3)
    X_test = X_test.reshape(-1,3,imgsize[0],imgsize[1])
    
    return X_train, Y_train, X_test, Y_test

In [3]:
X_train, Y_train, X_test, Y_test = loadData()

In [5]:
os.environ['KERAS_BACKEND']='tensorflow'
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,Adam
from keras.layers.advanced_activations import PReLU
from keras.callbacks import ModelCheckpoint

model = Sequential()
# input: 1080x1920 images with 3 channels -> (3, 1080, 1920) tensors.
# this applies 32 convolution filters of size 10x10 each.
model.add(Convolution2D(32, 10, 10, border_mode='valid', input_shape=(3, 1080, 1920))) #input image 1080x1920 screenshots 
model.add(Activation('relu'))
model.add(Convolution2D(32, 5, 5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.114))

model.add(Convolution2D(64, 5, 5, border_mode='valid'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Convolution2D(128, 3, 3, border_mode='valid'))
model.add(Activation('relu'))
model.add(Convolution2D(128, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.128))

model.add(Flatten())

model.add(Dense(512))
model.add(PReLU())
model.add(Dropout(0.35))
# Note: Keras does automatic shape inference.
model.add(Dense(256))
#model.add(Activation('relu'))
model.add(PReLU())
model.add(Dropout(0.35))

model.add(Dense(10))
model.add(Activation('softmax'))

# sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# checkpoint
filepath="models/genre.weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [6]:
model.fit(X_train, Y_train, validation_split=0.2, batch_size=32, nb_epoch=30, callbacks=callbacks_list)

Train on 33910 samples, validate on 8478 samples
Epoch 1/30
33910/33910 [==============================] - 137s - loss: 1.6995 - acc: 0.3737 - val_loss: 1.5760 - val_acc: 0.4237
Epoch 2/30
33910/33910 [==============================] - 136s - loss: 1.5889 - acc: 0.4240 - val_loss: 1.5520 - val_acc: 0.4285
Epoch 3/30
33910/33910 [==============================] - 136s - loss: 1.5444 - acc: 0.4387 - val_loss: 1.5167 - val_acc: 0.4463
Epoch 4/30
33910/33910 [==============================] - 136s - loss: 1.5163 - acc: 0.4444 - val_loss: 1.5032 - val_acc: 0.4475
Epoch 5/30
33910/33910 [==============================] - 136s - loss: 1.4913 - acc: 0.4538 - val_loss: 1.4854 - val_acc: 0.4565
Epoch 6/30
33910/33910 [==============================] - 136s - loss: 1.4678 - acc: 0.4593 - val_loss: 1.4585 - val_acc: 0.4584
Epoch 7/30
33910/33910 [==============================] - 136s - loss: 1.4517 - acc: 0.4680 - val_loss: 1.4785 - val_acc: 0.4540
Epoch 8/30
33910/33910 [========================

In [7]:
model.fit(X_train, Y_train, validation_split=0.2, batch_size=32, nb_epoch=20, callbacks=callbacks_list)

Train on 33910 samples, validate on 8478 samples
Epoch 1/20
33910/33910 [==============================] - 136s - loss: 0.8684 - acc: 0.6965 - val_loss: 1.2163 - val_acc: 0.6044
Epoch 2/20
33910/33910 [==============================] - 136s - loss: 0.8550 - acc: 0.6995 - val_loss: 1.2611 - val_acc: 0.5957
Epoch 3/20
33910/33910 [==============================] - 136s - loss: 0.8454 - acc: 0.7042 - val_loss: 1.2271 - val_acc: 0.6082
Epoch 4/20
33910/33910 [==============================] - 136s - loss: 0.8195 - acc: 0.7119 - val_loss: 1.2514 - val_acc: 0.6117
Epoch 5/20
33910/33910 [==============================] - 136s - loss: 0.8103 - acc: 0.7179 - val_loss: 1.2339 - val_acc: 0.6112
Epoch 6/20
33910/33910 [==============================] - 136s - loss: 0.7877 - acc: 0.7263 - val_loss: 1.2149 - val_acc: 0.6178
Epoch 7/20
33910/33910 [==============================] - 136s - loss: 0.7743 - acc: 0.7334 - val_loss: 1.1978 - val_acc: 0.6196
Epoch 8/20
33910/33910 [========================

In [8]:
model.fit(X_train, Y_train, validation_split=0.2, batch_size=32, nb_epoch=20, callbacks=callbacks_list)

Train on 33910 samples, validate on 8478 samples
Epoch 1/20
33910/33910 [==============================] - 136s - loss: 0.6427 - acc: 0.7802 - val_loss: 1.2449 - val_acc: 0.6425
Epoch 2/20
33910/33910 [==============================] - 137s - loss: 0.6347 - acc: 0.7846 - val_loss: 1.2404 - val_acc: 0.6472
Epoch 3/20
33910/33910 [==============================] - 137s - loss: 0.6223 - acc: 0.7894 - val_loss: 1.1967 - val_acc: 0.6537
Epoch 4/20
33910/33910 [==============================] - 136s - loss: 0.6052 - acc: 0.7941 - val_loss: 1.1851 - val_acc: 0.6505
Epoch 5/20
33910/33910 [==============================] - 136s - loss: 0.6115 - acc: 0.7941 - val_loss: 1.2084 - val_acc: 0.6531
Epoch 6/20
33910/33910 [==============================] - 136s - loss: 0.5957 - acc: 0.7989 - val_loss: 1.1757 - val_acc: 0.6623
Epoch 7/20
33910/33910 [==============================] - 136s - loss: 0.5798 - acc: 0.8050 - val_loss: 1.2165 - val_acc: 0.6539
Epoch 8/20
33910/33910 [========================

In [9]:
model.fit(X_train, Y_train, validation_split=0.2, batch_size=32, nb_epoch=20, callbacks=callbacks_list)

Train on 33910 samples, validate on 8478 samples
Epoch 1/20
33910/33910 [==============================] - 137s - loss: 0.5196 - acc: 0.8266 - val_loss: 1.2493 - val_acc: 0.6683
Epoch 2/20
33910/33910 [==============================] - 136s - loss: 0.5116 - acc: 0.8299 - val_loss: 1.1998 - val_acc: 0.6653
Epoch 3/20
33910/33910 [==============================] - 137s - loss: 0.5180 - acc: 0.8272 - val_loss: 1.2723 - val_acc: 0.6710
Epoch 4/20
33910/33910 [==============================] - 137s - loss: 0.5329 - acc: 0.8251 - val_loss: 1.2337 - val_acc: 0.6748
Epoch 5/20
33910/33910 [==============================] - 136s - loss: 0.5089 - acc: 0.8321 - val_loss: 1.2346 - val_acc: 0.6662
Epoch 6/20
33910/33910 [==============================] - 137s - loss: 0.5104 - acc: 0.8297 - val_loss: 1.2793 - val_acc: 0.6699
Epoch 7/20
33910/33910 [==============================] - 136s - loss: 0.5010 - acc: 0.8355 - val_loss: 1.2779 - val_acc: 0.6688
Epoch 8/20
33910/33910 [========================

In [11]:
model.fit(X_train, Y_train, validation_split=0.2, batch_size=32, nb_epoch=20, callbacks=callbacks_list)

Train on 33910 samples, validate on 8478 samples
Epoch 1/20
33910/33910 [==============================] - 137s - loss: 0.4523 - acc: 0.8510 - val_loss: 1.1874 - val_acc: 0.6739
Epoch 2/20
33910/33910 [==============================] - 136s - loss: 0.4709 - acc: 0.8460 - val_loss: 1.2663 - val_acc: 0.6736
Epoch 3/20
33910/33910 [==============================] - 137s - loss: 0.4764 - acc: 0.8432 - val_loss: 1.2538 - val_acc: 0.6702
Epoch 4/20
33910/33910 [==============================] - 136s - loss: 0.4512 - acc: 0.8520 - val_loss: 1.2945 - val_acc: 0.6733
Epoch 5/20
33910/33910 [==============================] - 137s - loss: 0.4511 - acc: 0.8531 - val_loss: 1.3168 - val_acc: 0.6756
Epoch 6/20
33910/33910 [==============================] - 137s - loss: 0.4655 - acc: 0.8485 - val_loss: 1.1675 - val_acc: 0.6806
Epoch 7/20
33910/33910 [==============================] - 136s - loss: 0.4570 - acc: 0.8519 - val_loss: 1.2821 - val_acc: 0.6787
Epoch 8/20
33910/33910 [========================

KeyboardInterrupt: 

In [12]:
score = model.evaluate(X_test, Y_test, batch_size=32)
print score

10592/10597 [============================>.] - ETA: 0s[1.2369223987537588, 0.67537982448143841]
